In [3]:
from importlib import import_module
from torch.utils.data import DataLoader
from train_eval import test
from utils import CustomDataset, DataConfig


In [9]:
def call(model_path, model_name='TextCNN', dataset='ship_data',
         embedding='embedding.npz'):
    model_module = import_module(f'models.{model_name}')
    model_config = model_module.Config()
    data_config = DataConfig(dataset, embedding)
    test_dataset = CustomDataset(data_config, data_class='test')
    test_loader = DataLoader(test_dataset, batch_size=model_config.batch_size)
    model = model_module.Model(model_config, data_config).to(data_config.device)
    test(data_config, model, test_loader, model_path)

In [10]:
call(model_path='./result/TextCNN_单层嵌入层.ckpt')

Test Loss:  0.19,  Test Acc: 93.92%
Precision, Recall and F1-Score...
              precision    recall  f1-score   support

        散杂货船     0.9092    0.9528    0.9305     44333
          渔船     0.9736    0.9855    0.9795     40276
        集装箱船     0.9330    0.9157    0.9243     18656
          油船     0.9562    0.8018    0.8722      6458
       液体散货船     0.9420    0.7466    0.8330      5919

    accuracy                         0.9392    115642
   macro avg     0.9428    0.8805    0.9079    115642
weighted avg     0.9398    0.9392    0.9383    115642

Confusion Matrix...
[[42240   862   950   141   140]
 [  552 39691    15    12     6]
 [ 1461    59 17084     6    46]
 [ 1080    88    32  5178    80]
 [ 1127    66   229    78  4419]]


In [21]:
import torch
import torch.nn as nn
import torchvision
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        self.feature_extraction = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=96, kernel_size=3, stride=1, padding=1, bias=False),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2), 
            nn.Conv2d(in_channels=96, out_channels=192, kernel_size=3, stride=1, padding=1, bias=False),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2), 
            nn.Conv2d(in_channels=192, out_channels=384, kernel_size=3, stride=1, padding=1, bias=False),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, stride=1, padding=1, bias=False),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1, bias=False),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(in_features=256 * 3 * 3, out_features=4096),  # 修改in_features以适应输入尺寸
            nn.Dropout(p=0.5),
            nn.Linear(in_features=4096, out_features=4096),
            nn.Linear(in_features=4096, out_features=4),
        )

    def forward(self, x):
        x = self.feature_extraction(x)
        x = x.view(x.size(0), 256 * 3 * 3)  # 修改reshape的参数以适应输入尺寸
        x = self.classifier(x)
        return x


In [23]:
import torch
from torchvision import transforms
from PIL import Image
transform = transforms.Compose([
    transforms.ToTensor(),           # 将图像转换为张量
])

# 2. 读取图像
image = Image.open("./ship_data/pic_data/mmsi_50.png")  # 替换"example.jpg"为你的图片路径

# 3. 数据转换
image = transform(image)
# 添加一维
image = image.unsqueeze(0)  # 在第0维度添加一维

model=AlexNet()
a=model(image)

# 5. 显示图像张量的大小
print(a)

tensor([[-0.0064, -0.0140,  0.0149,  0.0145]], grad_fn=<AddmmBackward0>)


In [24]:
import argparse
import pickle as pkl
from datetime import datetime

import numpy as np
import torch
from torch.utils.data import DataLoader
from torchinfo import summary


import models.ShipRNN as ShipRNN
from train_eval import train, init_network, test
from utils import CustomDataset, DataConfig
# 随机种子设置
np.random.seed(3407)
torch.manual_seed(3407)
torch.cuda.manual_seed_all(3407)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

embedding = 'word2vec'
dim = 50
model_name = 'ShipRNN'
class_type = 'cluster'

model_config = ShipRNN.ModelConfig(notes='')
data_config = DataConfig(embedding, dim, class_type)
print(model_name, data_config.dim, embedding)

# 创建自定义数据集
print('start read data...')
train_dataset = CustomDataset(data_config, data_class='train')
val_dataset = CustomDataset(data_config, data_class='val')
test_dataset = CustomDataset(data_config, data_class='test')
vocab = pkl.load(open(data_config.vocab_path, 'rb'))
data_config.n_vocab = len(vocab)
print('read data done...')



ShipRNN 50 word2vec
start read data...
read data done...


In [25]:
# 数据加载器
train_loader = DataLoader(train_dataset, batch_size=data_config.batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=data_config.batch_size)
test_loader = DataLoader(test_dataset, batch_size=data_config.batch_size)



In [30]:
import models.AlexNet as AlexNet
# model = ShipRNN.Model(model_config, data_config).to(data_config.device)
model = AlexNet.AlexNet(data_config)

# 初始化模型参数

init_network(model)
summary(model, input_size=(2, 1, 31, 28), dtypes=[torch.float])


Layer (type:depth-idx)                   Output Shape              Param #
AlexNet                                  [2, 5]                    --
├─Sequential: 1-1                        [2, 256, 3, 3]            --
│    └─Conv2d: 2-1                       [2, 96, 31, 28]           864
│    └─ReLU: 2-2                         [2, 96, 31, 28]           --
│    └─MaxPool2d: 2-3                    [2, 96, 15, 14]           --
│    └─Conv2d: 2-4                       [2, 192, 15, 14]          165,888
│    └─ReLU: 2-5                         [2, 192, 15, 14]          --
│    └─MaxPool2d: 2-6                    [2, 192, 7, 7]            --
│    └─Conv2d: 2-7                       [2, 384, 7, 7]            663,552
│    └─ReLU: 2-8                         [2, 384, 7, 7]            --
│    └─Conv2d: 2-9                       [2, 256, 7, 7]            884,736
│    └─ReLU: 2-10                        [2, 256, 7, 7]            --
│    └─Conv2d: 2-11                      [2, 256, 7, 7]            58

In [32]:
train(model_config, data_config, model, train_loader, val_loader, '')
# 将测试结果写入文件


Adjusting learning rate of group 0 to 1.0000e-03.
Epoch [1/100]


ValueError: too many values to unpack (expected 2)

In [ ]:
res, res1 = test(data_config, model, test_loader, model_path=model_config.save_path)

# 获取当前时间
current_time = datetime.now()
formatted_time = current_time.strftime("%Y-%m-%d %H:%M")

with open(f'res_6/{model_config.model_name}_{notes}_{formatted_time}_{embedding}.txt', "w") as file:
    file.write(str(res))
    file.write(str(res1))